In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from isabelle_connector.isabelle_connector import IsabelleConnector

In [3]:
isabelle = IsabelleConnector(
    name="extract_transitions", working_directory=".", debug=True
)

### Missing Isabelle component: "/Users/yousef/.isabelle/contrib/hugo-0.119.0"


In [7]:
from argparse import Namespace

from isabelle_connector.config import HOL_DIR, PROJ_ROOT

configs = Namespace(
    # dataset_name="yalhessi/lemexp",
    # dataset_config="hol-thms",
    root_dir=HOL_DIR,
    imports=[
        PROJ_ROOT / "isabelle-thys/ExtractLemmas",
        PROJ_ROOT / "isabelle-thys/RoughSpec",
    ],
    # upload=True,
)

In [8]:
from isabelle_connector.extraction import transitions_theory
from isabelle_connector.isabelle_utils import get_theory, list_theory_files


thy_files = list_theory_files(configs.root_dir)
src_thys = [get_theory(theory_file, configs.root_dir) for theory_file in thy_files]
transition_thys = [transitions_theory(thy, configs) for thy in src_thys]


In [9]:
data, errs = isabelle.use_theories(transition_thys, batch_size=1000, recache=True)

Using cached results for 0 / 1462 theories
Adding new session for HOL after 2000 theories


DONE:   0%|          | 0/2 [00:00<?, ?it/s]

Successful values from 1462 / 1462 theories
func:use_theories took: 49.11764907836914 sec


In [10]:
# data["Transitions_ATP"]

In [ ]:
from isabelle_connector.extraction import template_and_type_extraction_theory


extraction_thys = [
    template_and_type_extraction_theory(thy, configs) for thy in src_thys
]

In [13]:
data, errs = isabelle.use_theories(extraction_thys, batch_size=1)

Using cached results for 0 / 1462 theories
Starting session 1 / 78: HOL-Prolog
Starting session 2 / 78: HOLCF-IMP
Starting session 3 / 78: HOL-Types_To_Sets
Starting session 4 / 78: HOL-Predicate_Compile_Examples
Starting session 5 / 78: HOL-SPARK-Manual
Starting session 6 / 78: HOL-Real_Asymp-Manual
Starting session 7 / 78: HOL-Isar_Examples
Starting session 8 / 78: HOL-Mutabelle
Starting session 9 / 78: HOL-ZF
Starting session 10 / 78: HOL-MicroJava
Starting session 11 / 78: HOL-IMPP
Starting session 12 / 78: IOA-Storage
Starting session 13 / 78: HOL-IMP
Starting session 14 / 78: HOL-Probability-ex
Starting session 15 / 78: HOL-Quotient_Examples
Starting session 16 / 78: HOL-Proofs-ex


KeyboardInterrupt: 